In [1]:
import numpy as np


In [8]:
import os.path as osp
import cv2
import numpy as np

import sys
sys.path.append('../../../') 

from dust3r.datasets.base.base_stereo_view_dataset import BaseStereoViewDataset
from dust3r.utils.image import imread_cv2


In [63]:
from itertools import combinations
from collections import defaultdict
from tqdm.auto import tqdm

def compute_iou(pts1, pts2):
    """
    Compute the Intersection-over-Union (IoU) of two sets of 3D points.
    """
    pts1 = np.asarray(pts1)
    pts2 = np.asarray(pts2)
    
    # Find common points (intersection)
    intersection = np.sum(np.all(pts1[:, None, :] == pts2[None, :, :], axis=2))

    # Find unique points (union)
    combined = np.vstack((pts1, pts2))
    union = np.unique(combined, axis=0).shape[0]
    
    # Compute IoU
    return intersection / union if union > 0 else 0

def quality_pair_score(iou, alpha):
    """
    Compute the quality pair score s = IoU × 4 cos(α)(1 - cos(α))
    """
    angle_term = 4 * np.cos(alpha) * (1 - np.cos(alpha))
    return iou * angle_term if angle_term > 0 else 0

def select_best_pairs(dataset, iou_threshold=0.75, score_threshold=0.1, pairs_per_image=5, pairs_number=10):
    """
    Select the best image pairs using a greedy algorithm
    """
    pairs = list(combinations(dataset, 2))
    pair_scores = []

    for img1, img2 in pairs[:100]:
        img1 = img1[0]
        img2 = img2[0]
        
        print(img1['idx'], img2['idx'])
        
        iou = compute_iou(img1['pts3d'], img2['pts3d'])
        print('IOU', iou)
        if iou == 0:
            continue
        
        pose1, pose2 = img1['camera_pose'], img2['camera_pose']
        rotation_diff = pose1[:3, :3].T @ pose2[:3, :3]  # Relative rotation matrix
        alpha = np.arccos(np.clip((np.trace(rotation_diff) - 1) / 2, -1, 1))  # Angle in radians
        
        score = quality_pair_score(iou, alpha)
        print('SCORE', img1['idx'], img2['idx'], score)
        if score > score_threshold:
            pair_scores.append((score, img1, img2))

    pair_scores.sort(key=lambda x: x[0], reverse=True)

    selected_pairs = []
    used_images = set()

    for score, img1, img2 in pair_scores:
        if img1['idx'] in used_images or img2['idx'] in used_images:
            continue

        selected_pairs.append((img1, img2))
        used_images.add(img1['idx'])
        used_images.add(img2['idx'])

        for img in dataset:
            if img['idx'] not in used_images:
                if compute_iou(img['pts3d'], img1['pts3d']) > iou_threshold or \
                   compute_iou(img['pts3d'], img2['pts3d']) > iou_threshold:
                    used_images.add(img['idx'])
                    
    # used_images = defaultdict(int)
    # 
    # for score, img1, img2 in pair_scores:
    #     if used_images[img1['idx']] > pairs_per_image or used_images[img2['idx']] > pairs_per_image:
    #         continue
    # 
    #     selected_pairs.append((img1, img2))
    #     used_images[img1['idx']] += 1
    #     used_images[img2['idx']] += 1

    # if len(selected_pairs) < pairs_number: 
    #     for i in range(len(dataset) - 1):
    #         if len(selected_pairs) >= pairs_number:
    #             break
    #         img1, img2 = dataset[i], dataset[i + 1]
    #         selected_pairs.append((img1, img2))

    return selected_pairs

selected_pairs = select_best_pairs(dataset)
for img1, img2 in selected_pairs:
    print(f"Selected pair: Image {img1['idx']} and Image {img2['idx']}")

(0, 0, 0) (1, 0, 0)
IOU 0.0
(0, 0, 0) (2, 0, 0)
IOU 0.0
(0, 0, 0) (3, 0, 0)
IOU 0.0
(0, 0, 0) (4, 0, 0)
IOU 0.0
(0, 0, 0) (5, 0, 0)
IOU 0.0
(0, 0, 0) (6, 0, 0)
IOU 0.0
(0, 0, 0) (7, 0, 0)
IOU 0.0
(0, 0, 0) (8, 0, 0)
IOU 0.0
(0, 0, 0) (9, 0, 0)
IOU 0.0
(0, 0, 0) (10, 0, 0)
IOU 0.0
(0, 0, 0) (11, 0, 0)
IOU 0.0
(0, 0, 0) (12, 0, 0)
IOU 0.0
(0, 0, 0) (13, 0, 0)
IOU 0.0
(0, 0, 0) (14, 0, 0)
IOU 0.0
(0, 0, 0) (15, 0, 0)
IOU 0.0
(0, 0, 0) (16, 0, 0)
IOU 0.0
(0, 0, 0) (17, 0, 0)
IOU 0.0
(0, 0, 0) (18, 0, 0)
IOU 0.0
(0, 0, 0) (19, 0, 0)
IOU 0.0
(0, 0, 0) (20, 0, 0)
IOU 0.0
(0, 0, 0) (21, 0, 0)
IOU 0.0
(0, 0, 0) (22, 0, 0)
IOU 0.0
(0, 0, 0) (23, 0, 0)
IOU 0.0
(0, 0, 0) (24, 0, 0)
IOU 0.0
(0, 0, 0) (25, 0, 0)
IOU 0.0
(0, 0, 0) (26, 0, 0)
IOU 0.0
(0, 0, 0) (27, 0, 0)
IOU 0.0
(0, 0, 0) (28, 0, 0)
IOU 0.0
(0, 0, 0) (29, 0, 0)
IOU 0.0
(0, 0, 0) (30, 0, 0)
IOU 0.0
(0, 0, 0) (31, 0, 0)
IOU 0.0
(0, 0, 0) (32, 0, 0)
IOU 0.0
(0, 0, 0) (33, 0, 0)
IOU 0.0
(0, 0, 0) (34, 0, 0)
IOU 0.0
(0, 0, 0) (35, 0, 0)
IO

In [61]:
selected_pairs

[]

In [60]:
for img1, img2 in selected_pairs:
    # print(img1[0])
    print(f"Selected pair: Image {img1[0]['idx']} and Image {img2[0]['idx']}")

In [10]:
from dust3r.datasets import UnderWaterDataset

In [51]:
dataset = UnderWaterDataset(split='test', ROOT='/home/aleksandra/dense_glomap_output', resolution=224)

In [23]:
dataset[0][0].keys()

dict_keys(['img', 'depthmap', 'camera_pose', 'camera_intrinsics', 'dataset', 'label', 'instance', 'idx', 'true_shape', 'pts3d', 'valid_mask', 'rng'])

In [42]:
selected_pairs = select_best_pairs(dataset)
for img1, img2 in selected_pairs:
    print(f"Selected pair: Image {img1['idx']} and Image {img2['idx']}")

float32 float32


TypeError: unhashable type: 'numpy.ndarray'